## IMD 0190 - TÓPICOS ESPECIAIS EM BUSINESS INTELIGENCE E ANALYTICS

#### Fernando Lucas, Renata Gurgel e Victor Gabriel

We based our work on notebooks that can be accessed at: https://github.com/eliasjacob/deep_learning_gen_ai/blob/main/

#### Keypoints

- Utilization of the free *GEMINI* and *llama* APIs as LLMs for project development.  

### Summary  

- **1st Project:** Development of an agent capable of classifying course evaluations offered by the Electoral Judiciary School of Rio Grande do Norte (EJE/RN).  

- **2nd Project:** Development of an AI agent capable of answering users' questions regarding the data available on the Open Data Portal of the Regional Electoral Court of Rio Grande do Norte (TRE/RN). The aforementioned portal can be accessed through the link: https://dados.tre-rn.jus.br/group/pessoas.  

- **3rd Project:** Utilization of an RAG structure to create a conversational AI agent based on the data from the open data portal. This is a different approach with the same theme as the 2nd Project.  


### Importar bibliotecas

In [26]:
!pip install langchain_experimental langchain requests langchain-community langchain_google_genai google-generativeai langchain_ollama ollama

In [31]:
import pandas as pd

### Zero-Shot Text Classification - An Real Application to Escola Judiciária Eleitoral do Rio Grande do Norte (EJE/RN)

#### Keypoints

- This is a init for a reviews courses classifications using the method of Zero-Shot Text Classification

- We was incrementing the project step-by-step. So, some codes can be repetitive with different approaches

- The first implementation we have defined the schema using reviews segmented in *Positive*, *Negative* and *Neutral* generated by `DeepSeek-V3`. Then, use the `.with_structured_method` to format the output response about the sentiment.

#### Import Libraries

In [ ]:
import os

In [ ]:
pip install openpyxl pandas

#### Defining the LLM models

In [ ]:
# Import ChatOllama class from langchain_ollama module
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI
import os

gemini_api_key = os.getenv("GEMINI_API_KEY")
# insert your gemini api key here
gemini_api_key = ""
gemini = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=gemini_api_key,
    temperature=0.0
    )

# Initialize the ChatOllama model with specific parameters
model_llama = ChatOllama(
    model="llama3.1",  # Specify the model to use
    base_url="http://localhost:11434",  # Set the base URL where the Ollama service is running
    temperature=0.0,  # Set the temperature for response variability
)

#### Lists of reviews

- We used *DeepSeek-V3* to generate 10 feedbacks for each of the possible classifications: positive, negative, and neutral.

- These lists were created to provide representative examples of each type of sentiment in the reviews.

- Later, for testing purposes, an *unlabeled feedback* list was created, containing all the reviews from the previous lists.


In [ ]:
# This is a example list of negative feedbacks
negative_feedbacks = [
    "O conteúdo é muito superficial, não aprofunda os temas.",
    "As aulas são longas e cansativas, difícil manter a atenção.",
    "Faltou material prático para aplicar o que foi ensinado.",
    "A plataforma é lenta e trava constantemente.",
    "O instrutor não explica de forma clara, fica confuso.",
    "O curso é caro para o pouco conteúdo que oferece.",
    "Não gostei da falta de interação com os instrutores.",
    "O certificado não é reconhecido no mercado.",
    "As atividades são repetitivas e pouco desafiadoras.",
    "O suporte é demorado e não resolve os problemas direito."
]
# This is a example list of positive feedbacks
positive_feedbacks = [
    "O curso é excelente, conteúdo muito bem organizado!",
    "Aprendi muito e consegui aplicar no meu trabalho. Recomendo!",
    "Instrutores claros e didáticos. Foi uma ótima experiência.",
    "O material de apoio é completo e fácil de entender.",
    "Adorei a plataforma, muito intuitiva e fácil de usar.",
    "O curso superou minhas expectativas. Parabéns à equipe!",
    "Conteúdo atualizado e relevante para o mercado atual.",
    "As atividades práticas ajudaram a fixar o conhecimento.",
    "Suporte rápido e eficiente. Tive todas as minhas dúvidas resolvidas.",
    "A interação com os alunos foi excelente."
]

# This is a example list of neutral feedbacks
neutral_feedbacks = [
    "O curso é bom, mas poderia ter mais exemplos práticos.",
    "O conteúdo é interessante, mas algumas aulas são muito longas.",
    "Gostei do material, mas a plataforma poderia ser mais estável.",
    "O curso atendeu às minhas expectativas, mas nada extraordinário.",
    "As explicações são claras, mas o ritmo é um pouco lento.",
    "O conteúdo é útil, mas senti falta de mais interação com os instrutores.",
    "O curso é razoável, mas o preço poderia ser mais acessível.",
    "Achei o material completo, mas algumas partes são repetitivas.",
    "O curso é bom para iniciantes, mas avançados podem achar básico.",
    "A experiência foi ok, mas esperava mais atividades práticas."
]

# This is a example list of feedbacks not classified
unlabelled_feedbacks = positive_feedbacks + negative_feedbacks + neutral_feedbacks

#### Response Structure

- Definition of the `SentimentAnalysisResponse` class, which will be used as the structural basis for storing the results of sentiment analysis.

- The class inherits from `BaseModel` in Pydantic.

- The `sentiment` attribute represents the sentiment label assigned to the text, which can only take the values "positive," "neutral," or "negative."

- The `Field` is used to define a description of the field, aiding in documentation and validation of the allowed values.

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal


class SentimentAnalysisResponse(BaseModel):
    """The response of a function that performs sentiment analysis on text."""

    # The sentiment label assigned to the text
    sentiment: Literal["positive", "neutral", "negative"] = Field(
        default_factory=str,
        description="The sentiment label assigned to the text. You can only have 'positive' or 'negative' as values.",
    )

#### Prompting the model

- We use the `.with_structured_output` method, which allows structuring the model's output in a more organized way.

- It acts as a response parser, ensuring that the output is in the expected format using the `SentimentAnalysisResponse` class.

- In the example below, we pass a generic textual review about a class so that the model can determine its sentiment.

In [ ]:
model_sentiment_gemini = gemini.with_structured_output(SentimentAnalysisResponse)

output = model_sentiment_gemini.invoke("A aula não foi muito boa e as atividades não foram interessantes como o esperado.")
output

- Conclusion: quickly and with little development effort, it is possible to obtain an application that solves a real problem for people who really need it.

#### Implementing the Agent

- Initially, we created two tools: one responsible for responding with the analyzed evaluation and its classification, and another responsible for counting the total occurrences of each class.

- After discussing with the client, we realized that it made more sense to centralize everything into a single tool. This tool can respond with a classification for each evaluation when receiving a list of comments and provide the total count. If it receives only one comment, it will return only the classification.

- Their names are: `analyze_sentiment()` and `count_sentiments()`.

- We used the *LangChain* function `create_tool_calling_agent` to create the agent.

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Literal, Union
from langchain.tools import tool
from langchain.memory import ConversationBufferMemory
from collections import Counter
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

class SentimentAnalysisResponse(BaseModel):
    """The response of a function that performs sentiment analysis on text."""

    # The sentiment label assigned to the text
    sentiment: Literal["positive", "neutral", "negative"] = Field(
        default_factory=str,
        description="The sentiment label assigned to the text. You can only have 'positive', 'neutral' or 'negative' as values.",
    )
model_sentiment_gemini = gemini.with_structured_output(SentimentAnalysisResponse)
# Criar um Tool para análise de sentimentos
@tool
def analyze_sentiment(texts: Union[str, List[str]]) -> dict:
    """
    Recebe um único texto ou uma lista de textos e retorna a classificação de sentimento de cada um.
    Se for uma lista, também retorna a contagem total de cada classificação.
    """

    if isinstance(texts, str):
        # Caso seja um único texto, retorna apenas sua classificação
        return model_sentiment_gemini.invoke(texts)

    elif isinstance(texts, list):
        # Caso seja uma lista, processa cada um e gera um resumo da contagem
        sentiment_counts = Counter()
        individual_results = []

        for text in texts:
            result = model_sentiment_gemini.invoke(text)
            sentiment_counts[result.sentiment] += 1
            individual_results.append(result)

        return {
            "individual_results": individual_results,
            "total_counts": dict(sentiment_counts)
        }


# Criar um Tool para contar os sentimentos
@tool
def count_sentiments(texts: List[str]) -> dict:
    """Recebe uma lista de textos e retorna a contagem de sentimentos (positivo, neutro, negativo)."""
    sentiment_counts = Counter()

    for text in texts:
        output = model_sentiment_gemini.invoke(text)
        sentiment_counts[output.sentiment] += 1

    return dict(sentiment_counts)

# Criar um agente com memória
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Define a prompt template for the chat, including system instructions and placeholders
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
tools=[analyze_sentiment]
# Inicializar o agente
agent = create_tool_calling_agent(
    tools=tools,  # Tools disponíveis para o agente
    llm=gemini,
    prompt=prompt
)

- The data source comes from Google Forms, and after all students have responded, the data can be downloaded in various formats, including **CSV**.

- Therefore, we import the dataframe and convert it into a list containing only non-null reviews.

In [ ]:
import pandas as pd

# Configuração da fonte de dados
sheet_id = ""
sheet_name = ""  # Certifique-se de usar o nome correto da aba

# URL gerada pelo Google Sheets para exportação como CSV
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

# Carregar os dados no Pandas
#df_drive = pd.read_csv(url, header=None, usecols=[14], dtype=str)
df_manual = pd.read_csv("./Avaliação de Reação.csv", usecols=[14])

reviews = df_manual.dropna()
reviews.head(50)
reviews_filtered = reviews.head()
unlabelled_feedbacks = reviews_filtered['Unnamed: 14'].tolist()
print(len(unlabelled_feedbacks))


- After defining the agent, we can use `AgentExecutor()` from *LangChain* to execute the agent, receiving user input.

- We use 5 previously unclassified data points obtained from the dataframe inserted earlier.

In [ ]:
# Create an instance of AgentExecutor with specified agent, tools, and verbosity
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Invoke the agent_executor with a dictionary containing the input query
output = agent_executor.invoke(
    {
        "input": f"Informe a classificação para cada um dos textos em {reviews_filtered['Unnamed: 14']}",  # The input query asking for the price of Bitcoin
    }
)
print(output['output'])

- Implementation of the interface in Streamlit running locally.

In [ ]:
!streamlit run app.py --server.port=8501 --browser.serverAddress=127.0.0.1

### Open Data Portal - Pandas Agent

#### Keypoints

- The open data portal provides transparency data from TRE/RN, grouped into major areas such as: Personnel Management Secretariat, Information Technology and Elections Secretariat, and Electoral Zones.

- The portal allows data to be downloaded in CSV format without the need for a request—simply access the website and select the topic of interest. The website is available at: [https://dados.tre-rn.jus.br/](https://dados.tre-rn.jus.br/).

- The project consists of implementing an artificial intelligence agent to answer questions and perform initial analyses on the dataframes.

In [ ]:
# Import the function to create an agent that works with pandas DataFrames
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
# Import the pandas library for data manipulation
import pandas as pd

# Read the Servidor_Ativo dataset from the provided URL into a pandas DataFrame
#df = pd.read_csv("/content/servidor_ativo.csv", delimiter=";")
# Read the Ex_Servidor dataset from the provided URL into a pandas DataFrame
#df1 = pd.read_csv("/content/ex_servidor.csv", delimiter=";")
# Read the Servidor_Inativo dataset from the provided URL into a pandas DataFrame
#df2 = pd.read_csv("/content/servidor_inativo.csv", delimiter=";")
# Read the Servidor_Ausente dataset from the provided URL into a pandas DataFrame
#df3 = pd.read_csv("/content/servidor_ausente.csv", delimiter=";")
# Read the Servidor_Instituidor_Pensao dataset from the provided URL into a pandas DataFrame
#df4 = pd.read_csv("/content/servidor_instituidor_pensao.csv", delimiter=";")
df_diarias = pd.read_csv("./diarias-historico.csv", delimiter=";")
df_passagens = pd.read_csv("./passagens-historico.csv", delimiter=";")

# Display the first few rows of the DataFrame to verify the data has been loaded correctly
df_diarias.head()

In [ ]:
df_passagens.head()

In [ ]:
df_passagens['COMPANHIA'].nunique()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

import os

gemini_api_key = ""
gemini = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=gemini_api_key,
    temperature=0.0
    )

In [ ]:
# Criar um PromptTemplate para reformular as respostas de forma mais conversacional
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="""
    Você é um assistente amigável e prestativo. Sua tarefa é reformular a resposta fornecida de maneira clara, explicativa e simpática. 
    A resposta deve ser dada em Português-Brasil e deve soar natural, como se fosse uma conversa.

    Resposta original: {query}

    Resposta reformulada:
    """
)
pipeline = prompt_template | gemini

In [ ]:
# Adicionar um prefixo ao prompt para explicar os dataframes
prefixo = """
Você tem acesso a dois dataframes:
1. `df_diarias`: Contém informações sobre diárias pagas.
2. `df_passagens`: Contém informações sobre passagens aéreas.

Use o dataframe apropriado para responder às perguntas.
"""
# Create a pandas dataframe agent using the specified LLM model and dataframe
pandas_agent = create_pandas_dataframe_agent(
    llm=gemini,  # Pass the initialized ChatOpenAI model
    df=[df_diarias, df_passagens],  # Provide the dataframe to be used by the agent
    verbose=True,  # Enable verbose mode for detailed logging
    agent_type="zero-shot-react-description",  # Specify the type of agent to create
    allow_dangerous_code=True,  # Opt-in to allow the use of the REPL tool
    prefix=prefixo
)

In [ ]:
def humanized_agent_response(query):
    try:
        # Obter a resposta original do pandas_agent
        resposta_original = pandas_agent.invoke(query)

        # Garantir que a resposta seja uma string
        if isinstance(resposta_original, dict):
            resposta_original = resposta_original.get("output", "") or resposta_original.get("input", "")
        resposta_original = str(resposta_original).strip()

        # Passar a resposta original para o pipeline para humanização
        resposta_humanizada = pipeline.invoke({"query": resposta_original})

        # Garantir que a resposta reformulada seja uma string
        if isinstance(resposta_humanizada, dict):
            resposta_humanizada = resposta_humanizada.get("output", "") or resposta_humanizada.get("query", "")
        resposta_humanizada = str(resposta_humanizada.content).strip()

        return resposta_humanizada

    except Exception as e:
        return f"Desculpe, ocorreu um erro ao reformular a resposta: {str(e)}"

In [ ]:
print(df_diarias['SERVIDOR'].nunique())

In [ ]:
query = "Pode me informar a quantidade de servidores diferentes?"
response = pandas_agent.invoke(query)
print(response)

#### Streamlit

### Open Data Portal - RAG

#### Installing the `FAISS`

- FAISS (Facebook AI Similarity Search) is a library developed by Facebook AI Research for efficient similarity search and clustering of high-dimensional vectors. It is widely used for tasks such as information retrieval, similarity search, and recommendation systems.

In [27]:
!pip install langchain faiss-cpu

#### CSV Loader
- This section demonstrates how to load a CSV file line by line as a collection of documents using `CSVLoader` from the `langchain.document_loaders` module. This is useful for processing structured data in natural language applications, such as Retrieval-Augmented Generation (RAG) and document indexing.

In [ ]:
from langchain.document_loaders import CSVLoader

# Caminho do arquivo
csv_path = "content\servidor_ativo.csv"
loader = CSVLoader(file_path="content\servidor_ativo.csv", csv_args={"delimiter":";"}, encoding='utf-8')
documentos = loader.load()
print(documentos)

[Document(metadata={'source': 'content\\servidor_ativo.csv', 'row': 0}, page_content='nome: ADARILIANY DE FRANÇA SILVA\nmatricula: 60002242\nlotacao: 04ª ZE\nnivel: \ncargo: \narea: \nespecialidade: \nsituacao: REQUISITADO\ncomissao: \ningresso: 2024-08-19'), Document(metadata={'source': 'content\\servidor_ativo.csv', 'row': 1}, page_content='nome: ADRIANA FERNANDES DE MEDEIROS\nmatricula: 92440643\nlotacao: AJCRE\nnivel: SUPERIOR\ncargo: ANALISTA JUDICIARIO\narea: ADMINISTRATIVA\nespecialidade: \nsituacao: EFETIVO\ncomissao: ASSISTENTE III\ningresso: 2006-03-09'), Document(metadata={'source': 'content\\servidor_ativo.csv', 'row': 2}, page_content='nome: ADRIANA KARLA DE OLIVEIRA FERREIRA BEZERRA\nmatricula: 30024390\nlotacao: NAI\nnivel: INTERMEDIÁRIO\ncargo: TECNICO JUDICIARIO\narea: APOIO ESPECIALIZADO\nespecialidade: ENFERMAGEM\nsituacao: EFETIVO\ncomissao: ASSISTENTE I\ningresso: 2006-02-16'), Document(metadata={'source': 'content\\servidor_ativo.csv', 'row': 3}, page_content='nom

#### Formatação dos documentos (linhas)

- This step reformats the documents extracted from the CSV into a continuous and structured textual format, making the information more understandable. The process transforms each line of the document into a readable text while preserving key attributes such as name, registration number, position, and department.

In [58]:
def reformat_document(doc):
    # Divide o conteúdo em linhas
    linhas = doc.page_content.split("\n")
    info = {}
    # Processa cada linha, extraindo chave e valor
    for linha in linhas:
        if ":" in linha:
            chave, valor = linha.split(":", 1)
            # Se o valor estiver vazio após remover espaços, atribua "não informado"
            valor_formatado = valor.strip() if valor.strip() else "não informado"
            info[chave.strip()] = valor_formatado
    # Cria um texto formatado de forma natural
    texto_formatado = (
        f"{info.get('nome', 'não informado')}, matrícula {info.get('matricula', 'não informado')}, "
        f"lotado em {info.get('lotacao', 'não informado')}, nível {info.get('nivel', 'não informado')}, "
        f"cargo {info.get('cargo', 'não informado')}, área {info.get('area', 'não informado')}, "
        f"situação {info.get('situacao', 'não informado')}, especialista em {info.get('especialidade', 'não informado')} ingresso em {info.get('ingresso', 'não informado')}."
    )
    # Atualiza o conteúdo do documento
    #doc.page_content = texto_formatado
    return texto_formatado

# Aplica a reformatação a todos os documentos
documentos_formatados = [reformat_document(doc) for doc in documentos]
#documento = [reformat_document(doc) for doc in documentos]

In [59]:
print(documentos_formatados)

['ADARILIANY DE FRANÇA SILVA, matrícula 60002242, lotado em 04ª ZE, nível não informado, cargo não informado, área não informado, situação REQUISITADO, especialista em não informado ingresso em 2024-08-19.', 'ADRIANA FERNANDES DE MEDEIROS, matrícula 92440643, lotado em AJCRE, nível SUPERIOR, cargo ANALISTA JUDICIARIO, área ADMINISTRATIVA, situação EFETIVO, especialista em não informado ingresso em 2006-03-09.', 'ADRIANA KARLA DE OLIVEIRA FERREIRA BEZERRA, matrícula 30024390, lotado em NAI, nível INTERMEDIÁRIO, cargo TECNICO JUDICIARIO, área APOIO ESPECIALIZADO, situação EFETIVO, especialista em ENFERMAGEM ingresso em 2006-02-16.', 'ADRIANA YARA UCHOA BARRETO DE ARAUJO, matrícula 30024332, lotado em SECOP, nível INTERMEDIÁRIO, cargo TECNICO JUDICIARIO, área ADMINISTRATIVA, situação EFETIVO, especialista em não informado ingresso em 1994-07-04.', 'ADRIANO DE LIMA NOBREGA, matrícula 30024583, lotado em 68ª ZE, nível SUPERIOR, cargo TECNICO JUDICIARIO, área ADMINISTRATIVA, situação EFETIVO

- Uma abordagem diferente separando por chunks que não obteve resultados satisfatórios tendo em vista a independência entre cada linha do dataframe

In [61]:
'''# Definir o número de linhas por chunk
num_linhas_por_chunk = 5

# Dividir o documento único em linhas
linhas = documento.split("\n")

# Criar chunks de 50 linhas cada
chunks = ["\n".join(linhas[i : i + num_linhas_por_chunk]) for i in range(0, len(linhas), num_linhas_por_chunk)]

# Exibir informações sobre os chunks gerados
print(f"Total de chunks gerados: {len(chunks)}")
print("\n--- Exemplo de um Chunk ---\n")
print(chunks[0])  # Exibir o primeiro chunk para conferência'''


'# Definir o número de linhas por chunk\nnum_linhas_por_chunk = 5\n\n# Dividir o documento único em linhas\nlinhas = documento.split("\n")\n\n# Criar chunks de 50 linhas cada\nchunks = ["\n".join(linhas[i : i + num_linhas_por_chunk]) for i in range(0, len(linhas), num_linhas_por_chunk)]\n\n# Exibir informações sobre os chunks gerados\nprint(f"Total de chunks gerados: {len(chunks)}")\nprint("\n--- Exemplo de um Chunk ---\n")\nprint(chunks[0])  # Exibir o primeiro chunk para conferência'

#### Implementing the RAG

- Definição de chave da API do Google

In [62]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API key here")

- Utilização do modelo de embeddings do google

In [65]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [66]:
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
#documentos = [Document(page_content=chunk) for chunk in chunks]
vectorstore = FAISS.from_documents(documentos, embeddings)

In [ ]:
# Import ChatOllama class from langchain_ollama module
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI
import os

gemini_api_key = os.getenv("GEMINI_API_KEY")
# insert your gemini api key here
gemini_api_key = ""
gemini = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=gemini_api_key,
    temperature=0.0
    )

# Initialize the ChatOllama model with specific parameters
model_llama = ChatOllama(
    model="llama3.1",  # Specify the model to use
    base_url="http://localhost:11434",  # Set the base URL where the Ollama service is running
    temperature=0.0,  # Set the temperature for response variability
)

In [74]:
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

retriever = vectorstore.as_retriever(search_kwargs={"k": 8})
chain = load_qa_chain(gemini, chain_type="stuff")
qa_chain = RetrievalQA(combine_documents_chain=chain, retriever=retriever)

In [76]:
retrieved_docs = retriever.invoke("Me informe os servidores com ingresso exatamente no ano de 2010?")
for doc in retrieved_docs:
    print(doc.page_content)

nome: ERLON GONÇALVES DE BRITO ALMEIDA
matricula: 92440737
lotacao: 
nivel: SUPERIOR
cargo: ANALISTA JUDICIARIO
area: ADMINISTRATIVA
especialidade: 
situacao: EFETIVO REMOVIDO
comissao: 
ingresso: 2010-03-25
nome: ADRIANO FERNANDES DA SILVA
matricula: 60001780
lotacao: SOG
nivel: 
cargo: 
area: 
especialidade: 
situacao: EXERCÍCIO PROVISÓRIO
comissao: ASSISTENTE I
ingresso: 2013-04-02
nome: DIEGO MARINHEIRO CORDENONSE
matricula: 30024580
lotacao: 65ª ZE
nivel: SUPERIOR
cargo: TECNICO JUDICIARIO
area: ADMINISTRATIVA
especialidade: 
situacao: EFETIVO
comissao: ASSISTENTE I
ingresso: 2012-06-22
nome: JOSE ROBERTO OLIVEIRA DIAS
matricula: 60001692
lotacao: 10ª ZE
nivel: 
cargo: 
area: 
especialidade: 
situacao: REQUISITADO
comissao: ASSISTENTE I
ingresso: 2011-07-26
nome: DENILSON BASTOS DA SILVA
matricula: 20024241
lotacao: SSI
nivel: INTERMEDIÁRIO
cargo: TECNICO JUDICIARIO
area: ADMINISTRATIVA
especialidade: 
situacao: EFETIVO
comissao: CHEFE DE SEÇÃO
ingresso: 1998-05-14
nome: BENEDITA 

In [87]:
pergunta = "Qual o cargo da servidora BENEDITA e me diga se ela possui comissao."
resposta = qa_chain.invoke(pergunta)
print(resposta["result"])

BENEDITA BRITO DA SILVA é REQUISITADA e possui comissão de ASSISTENTE I.


In [86]:
pergunta = "Me informe os servidores com ingresso exatamente no ano de 2010?"
resposta = qa_chain.invoke(pergunta)
print(resposta["result"])

Os servidores com ingresso exatamente no ano de 2010 são:

*   ERLON GONÇALVES DE BRITO ALMEIDA, com ingresso em 2010-03-25
*   BENEDITA BRITO DA SILVA, com ingresso em 2010-07-21


In [79]:
df = pd.read_csv("content\servidor_ativo.csv", delimiter=";", encoding="UTF-8")

In [84]:
print(df.loc[df["ingresso"].str.contains("2010")])


                                   nome  matricula lotacao          nivel  \
67              BENEDITA BRITO DA SILVA   60001619  39ª ZE            NaN   
158    ERLON GONÇALVES DE BRITO ALMEIDA   92440737     NaN       SUPERIOR   
190  FRANCISCO DE ASSIS MARTINS CORREIA   60001589  05ª ZE            NaN   
232           HIRAN MEDEIROS DE AZEVEDO   60001626  68ª ZE            NaN   
339      LUIZ EDUARDO DA SILVA TEIXEIRA   60001602  52ª ZE            NaN   
417                OLAVO CORTEZ CEZARIO   92440779  13ª ZE       SUPERIOR   
422    PATRICIA TEIXEIRA BORGES E SOUZA   30024527   SEPAT  INTERMEDIÁRIO   
431            PRIMO VAZ DA COSTA FILHO   60001577   AJCRE            NaN   

                   cargo            area especialidade  \
67                   NaN             NaN           NaN   
158  ANALISTA JUDICIARIO  ADMINISTRATIVA                 
190                  NaN             NaN           NaN   
232                  NaN             NaN           NaN   
339             

#### Final Thoughts
- Working with dataframes to create a RAG may not be the best data structure.  

- Due to the natural independence of each row and the lack of strong relationships between the data.  

- Even so, for specific queries that request information from a single document, it is possible to obtain satisfactory responses.